In [1]:
#TRAINING IMAGES INPUT
import gzip
import numpy as np
import tensorflow as tf
import pandas as pd

f = gzip.open('train-images-idx3-ubyte.gz','r') #f is a file
image_size = 28  #originally 28
num_images = 60000

f.read(16)
buf = f.read(image_size * image_size * num_images)
data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
data = data.reshape(num_images, image_size, image_size, 1)
print(data.shape)
print(type(data))

(60000, 28, 28, 1)
<class 'numpy.ndarray'>


In [2]:
#TRAINING LABELS INPUT
f = gzip.open('train-labels-idx1-ubyte.gz','r') #f is a file
f.read(8)
z = f.read() #z is a bytes #z has all the labels
labels = []
for i in z:
    labels.append(i)
labels = np.array(labels)
#len(bites)
print(labels.size)

60000


In [3]:
#TEST IMAGES 
f = gzip.open('t10k-images-idx3-ubyte.gz','r') #f is a file
image_size = 28  #originally 28
num_images = 10000

f.read(16)
buf = f.read(image_size * image_size * num_images)
data1 = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
data1 = data1.reshape(num_images, image_size, image_size, 1)
print(data1.shape)

(10000, 28, 28, 1)


In [4]:
#TEST LABELS
f = gzip.open('t10k-labels-idx1-ubyte.gz','r') #f is a file
f.read(8)
z = f.read() #z is a bytes #z has all the labels
labels2 = []
for i in z:
    labels2.append(i)
labels2 = np.array(labels2)
len(labels2)

10000

In [5]:
#want to combine test and train data
#let's first combine the labels #bites
labels = labels.tolist()
len(labels)
labels2 = labels2.tolist()
len(labels2)
labels = labels + labels2
print(len(labels))

70000


In [6]:
#combining data arrays
data = data.tolist()
print("sup")
data1 = data1.tolist()
print("hi")
data = data + data1
print("hello")
data = np.array(data)
print(data.shape)

sup
hi
hello
(70000, 28, 28, 1)


In [8]:
#manipulating data array so that each row is a picture and columns are the bits
#print(data[0,:,:].shape)
#print(data[3,4:24,:,0]) #data[i,:,:,0] return ith matrix with blank header and footer
#data[i,4:-4,:,0] return ith matrix without footers

vectors = []
for matrix in range(70000):#70000
    vector = []
    for row in range(4,24):
        for col in range(0,28):
            vector.append(data[matrix,row,col,0])
    #print(vector)
    
    vectors.append(vector)
print("vectors ",end="")
print(len(vectors))
vectors = np.array(vectors)
print(vectors.shape)

vectors 70000
(70000, 560)


In [9]:
#Splitting the dataset into the Training set and Test set
X = data
y = labels
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(vectors, y, test_size = 0.2, random_state = 0)
print(X_train.shape)
print(X_test.shape)

(56000, 560)
(14000, 560)


In [11]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [16]:
# Training the Random Forest model on the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=100,criterion="entropy",random_state=0)
classifier.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [17]:
#predicting the test set results using our model
y_pred = classifier.predict(X_test)

In [18]:
#making confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1365    0    3    0    2    2    6    0    9    0]
 [   0 1561    3    3    3    1    1    4    2    2]
 [   3    3 1399    6    5    2    7    9    7    2]
 [   1    2   24 1357    0   18    0   14   17    2]
 [   3    0    1    0 1306    0    3    4    4   29]
 [   1    6    3   13    2 1173   15    0   11    7]
 [   5    2    1    0    3    7 1365    0    4    0]
 [   2    6   23    2   11    0    0 1394    2   18]
 [   1    7    5    5    5    7    3    0 1314   21]
 [   5    3    4   21   18    7    2   16    8 1277]]


0.9650714285714286